In [1]:

import spacy
import fr_core_news_sm
nlp = fr_core_news_sm.load()


import os
import csv
import numpy as np
import speech_recognition as sr
from scipy.sparse import csr_matrix
from sknetwork.path import shortest_path

from collections import defaultdict
import functools
import itertools

currentPath = os.path.dirname(os.path.abspath(''))

In [2]:
timeTableFileName = "/Users/mainabila/Desktop/EPITECH_PROJECTS/NLP_EPITECH/data_timetables_edited.csv"
pathCount = 0

In [3]:
trainStationNameToId = defaultdict(functools.partial(next, itertools.count()))

In [4]:
numberOfTrainstations = 0

In [5]:
with open(timeTableFileName, newline='', encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    # Set of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)

    # First reading to get the number different stations and init shape of trips object
    for row in reader:
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]
        # pprint.pprint(row)
        # print("idxA ---"  + str(idxA))
        # print("idxB ---"  + str(idxB))
    numberOfTrainstations = len(trainStationNameToId)
    print(numberOfTrainstations)
    trips = np.zeros((numberOfTrainstations, numberOfTrainstations))
    # pprint.pprint(trips)
    # Reset of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)
    # # Reading data
    for row in reader:
        pathCount += 1
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]

    #     # If trip already exist/has already been rode, display message
        indexTupple = (idxA, idxB) if idxA < idxB else (idxB, idxA)
        if trips[indexTupple] != 0:
            print(f"Trip {row[1]} - {row[2]} with a distance of {row[3]} has already be read with a distance of {trips[indexTupple]}. Ignoring the new one.")
        else:
            trips[indexTupple] = int(row[3])
        print(indexTupple , int(row[3]) )

817
(0, 1) 138
(1, 2) 145
(1, 3) 97
(1, 4) 194
(1, 5) 149
(1, 6) 147
(7, 8) 201
(7, 9) 195
(7, 10) 142
(11, 12) 122
(12, 13) 101
(11, 14) 131
(12, 15) 177
(11, 15) 38
(14, 15) 154
(15, 16) 146
(8, 17) 121
(12, 18) 75
(12, 19) 131
(18, 19) 72
(12, 20) 71
(14, 19) 146
(14, 18) 84
(12, 21) 118
(22, 23) 183
(23, 24) 145
(22, 25) 107
(22, 24) 28
(24, 25) 75
(23, 25) 74
(23, 26) 155
(23, 27) 119
(23, 28) 100
(27, 29) 80
(10, 17) 69
(5, 30) 111
(5, 19) 182
(9, 30) 135
(9, 31) 102
(9, 32) 70
(30, 32) 70
(19, 33) 38
(19, 34) 366
(15, 34) 211
(35, 36) 174
(36, 37) 88
(38, 39) 241
(38, 40) 97
(39, 40) 153
(39, 41) 172
(38, 41) 75
(38, 42) 247
(38, 43) 302
(41, 43) 216
(43, 44) 50
(43, 45) 159
(43, 46) 83
(42, 47) 74
(42, 48) 125
(42, 49) 181
(48, 49) 53
(49, 50) 277
(47, 49) 103
(47, 51) 82
(50, 51) 141
(43, 51) 246
(43, 50) 78
(14, 52) 171
(16, 52) 170
(51, 53) 262
(14, 51) 308
(51, 52) 124
(16, 51) 309
(14, 54) 262
(52, 54) 74
(16, 54) 263
(14, 34) 324
(34, 52) 137
(16, 34) 323
(51, 55) 217
(55

In [6]:
# Create dictionarry to map an id to its train station name
trainStationIdToName = dict((id, name) for name, id in trainStationNameToId.items())

In [7]:
# Make matrix symetrical as the trips are not directed but can be taken in both directions
# Source from https://stackoverflow.com/a/42209263
i_lower = np.tril_indices(numberOfTrainstations, -1)
trips[i_lower] = trips.T[i_lower]



In [8]:
# Creating Compressed Sparse Row (CSR) matrix to store and work more efficiently with only the trips
tripGraph = csr_matrix(trips)

In [9]:
tripGraph

<817x817 sparse matrix of type '<class 'numpy.float64'>'
	with 3150 stored elements in Compressed Sparse Row format>

In [10]:
print(f"Read {tripGraph.getnnz()} go and back trips ({int(tripGraph.getnnz() / 2)} distinc trips) out of {pathCount} rows for {len(trainStationNameToId)} distinct train stations.")

Read 3150 go and back trips (1575 distinc trips) out of 1575 rows for 817 distinct train stations.


In [11]:
class Trip:
    def __init__(self, startStationId, endStationId, path, totalDuration):
        self.startStationId = startStationId
        self.endStationId = endStationId
        self.path = path
        if totalDuration is None:
            self.totalDuration = None
        else:
            self.totalDuration = int(totalDuration)

    def __str__(self):
     return f"Trip from {self.startStationId} to {self.endStationId} for a total duration of {self.totalDuration} by this path: {self.path}"

In [12]:
# Functions used to determine the shortest path between cities

def getPathBetweenIds(trainStationStartIds: list, trainStationEndIds: list):
    global tripGraph
    paths = []
    # If start array contains one element also contained in end array -> return path from/to the same station
    for startId in trainStationStartIds:
        if startId in trainStationEndIds:
            return [int(startId), int(startId)]

    # As shortest_path() does not support multiple sources and multiple targets at the same time, we'll iterate through all start points and manually concat the results
    if(len(trainStationStartIds) > 1 and len(trainStationEndIds) > 1):
        for trainStationEndId in trainStationEndIds:
            results = shortest_path(tripGraph, sources=[int(i) for i in trainStationStartIds], targets=[int(trainStationEndId)], method='D')
            for result in results:
                if len(result) >= 2:
                    paths.append(result)
        return paths
    else:
        results = shortest_path(tripGraph, sources=[int(i) for i in trainStationStartIds], targets=[int(i) for i in trainStationEndIds], method='D')
        for result in results:
            if len(result) >= 2:
                paths.append(result)
        return paths

In [13]:
def isCityMatchingKey(city: str, key: str):
    return city.lower() in key.lower()

In [14]:

def getPathBetweenCities(start: str, end: str):
    trainStationStartIds = np.array([])
    trainStationEndIds = np.array([])

    # Get all stations that contains the searched name
    for key, value in trainStationNameToId.items():
        if isCityMatchingKey(start, key):
            trainStationStartIds = np.append(trainStationStartIds, value)
        if isCityMatchingKey(end, key):
            trainStationEndIds = np.append(trainStationEndIds, value)

    if len(trainStationStartIds) > 0 and len(trainStationEndIds) > 0:
        return getPathBetweenIds(trainStationStartIds, trainStationEndIds)
    else:
        return np.array([])

In [15]:
def getBestPathForFullTrip(tripCityWaypoints: list):
    global tripGraph
    fullTrip = np.array(np.zeros(len(tripCityWaypoints)-1), dtype=object)
    # Iterate through all sub trips
    for trip in range(len(fullTrip)):
        paths = getPathBetweenCities(tripCityWaypoints[trip], tripCityWaypoints[trip+1])
        minDistance = None
        keptPath = None
        startId = None
        endId = None

        # Iterate though the returned array
        for path in paths:
            distance = 0
            # Nested array => multiple start/end possible
            if isinstance(path, list):
                for i in range(len(path)-1):
                    distance = distance + tripGraph[(path[i], path[i+1])]
                if minDistance is None or distance < minDistance:
                    minDistance = distance
                    keptPath = path
                    startId = path[0]
                    endId = path[len(path)-1]

            # Scalar value => only one path possible
            else:
                for i in range(len(paths)-1):
                    distance = distance + tripGraph[(paths[i], paths[i+1])]
                minDistance = distance
                keptPath = paths
                startId = paths[0]
                endId = paths[len(paths)-1]

        fullTrip[trip] = Trip(startId, endId, keptPath, minDistance)
    return fullTrip



In [16]:
# Use following to test pathfinding functions above
def testPathfinding():
    bestTrips = getBestPathForFullTrip(['Orléan', 'Paris', 'Strasbourg', 'dsfdsfd', 'Mulhouse', 'Mulhouse'])
    for i in range(len(bestTrips)):
        if bestTrips[i].path is not None:
            print(f"#{i+1} - {bestTrips[i]}")
            for stationId in bestTrips[i].path:
                print(f"{trainStationIdToName[stationId]}")
        else:
            if bestTrips[i].startStationId is None or bestTrips[i].endStationId is None:
                print(f"#{i+1} - Could not find one or both station of the sub-trip")
            else:
                print(f"#{i+1} - Could not find a path between the both stations")
testPathfinding()

#1 - Trip from 18 to 12 for a total duration of 75 by this path: [18, 12]
Gare de Orléans
Gare de Paris-Austerlitz
#2 - Trip from 735 to 104 for a total duration of 287 by this path: [735, 669, 98, 104]
Gare de Paris-Est
Gare de Epernay
Gare de Metz-Ville
Gare de Strasbourg
#3 - Could not find one or both station of the sub-trip
#4 - Could not find one or both station of the sub-trip
#5 - Trip from 133 to 133 for a total duration of 0 by this path: [133, 133]
Gare de Mulhouse
Gare de Mulhouse


In [17]:
# Getting user's request
# Inspired from https://www.geeksforgeeks.org/python-convert-speech-to-text-and-text-to-speech/
try:
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting to noise level...")
        r.adjust_for_ambient_noise(source, duration=0.2)

        print("Listening...")
        audio = r.listen(source)
            
        print("Voice recognition...")
        request = r.recognize_google(audio, language="fr-FR")

        print(f"Parsed: '{request}'")

except sr.RequestError as e:
    print(f"Exception during request parsing: {e}")

Adjusting to noise level...
Listening...
Voice recognition...
Parsed: 'je veux aller de Paris en bateau'
